# Template

This notebook allows users to find common and specific genes in their experiment of interest using an *existing* VAE model (model trained by the user using `1_train_example.pynb`) and selecting a template experiment that is *not* included in the training compendium.

This notebook will generate a `generic_gene_summary_<experiment id>.tsv` file that contains z-scores per gene that indicates how specific a gene is the experiment in question.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from ponyo import utils, simulate_expression_data
from sophie import (
    process,
    stats,
    ranking,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


## Inputs

User needs to fill in the [config file](config_new_experiment.tsv) following the instructions provided in the [readme]](README.md)

In [3]:
# Read in config variables
config_filename = "config_example.tsv"

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Root directory containing analysis subdirectories and scripts
base_dir = params["base_dir"]

# Local directory to store intermediate files
local_dir = params["local_dir"]

# File containing un-normalized template experiment
raw_template_filename = params["raw_template_filename"]

# Un-normalized compendium filename
raw_compendium_filename = params["raw_compendium_filename"]

# Normalized compendium filename
normalized_compendium_filename = params["normalized_compendium_filename"]

# ID for template experiment to be selected
project_id = params["project_id"]

# Number of simulated experiments to generate
num_simulated = params["num_simulated"]

# Directory that simulated experiments will be written to
# This directory is created by https://github.com/greenelab/ponyo/blob/master/ponyo/utils.py
simulated_data_dir = params["simulated_data_dir"]

# Directory containing trained VAE model
vae_model_dir = params["vae_model_dir"]

# Size of the latent dimension
latent_dim = params["latent_dim"]

# Scaler transform used to scale compendium data into 0-1 range for training
scaler_transform_filename = params["scaler_transform_filename"]

# Which DE method to use
# We recommend that if data is RNA-seq then use DESeq2 ("deseq")
# If data is microarray then use Limma ("limma")
de_method = params["DE_method"]

# If using DE-seq, setting this parameter will
# remove genes below a certain threshold
count_threshold = params["count_threshold"]

# Metadata file that specifies which samples to keep for DE analysis (Optional)
# By default, a two-condition differential expression analysis is supported (case vs control).
# However, some experiments included more than 2 conditions and so these "extra" samples 
# should not considered in the downstream differential expression analysis. 
template_process_samples_filename = params["template_process_samples_filename"]
    
# Metadata file that specifies sample grouping for DE analysis
template_DE_grouping_filename = params["template_DE_grouping_filename"]

# Statistic to use to rank genes or pathways by
# Choices are "log2FoldChange" if using DESeq or "log2FC" 
# if using limma as the `de_method`
col_to_rank_genes = params["rank_genes_by"]

In [5]:
# Files generated by this notebook

# File storing template experiment with gene ids mapped to compendium gene ids
mapped_template_filename = params["mapped_template_filename"]

# File storing normalized template experiment
normalized_template_filename = params["normalized_template_filename"]

# File storing processed template experiment,
# after samples have been selected for comparison in DE analysis
processed_template_filename = params["processed_template_filename"]

# Output summary file
output_filename = params["output_filename"]

## Process template experiment

This step:
1. Normalizes the template experiment such that the template experiment and compendium experiment are in the same range
2. Ensures that the feature space (i.e. gene ids) are the same in the template and compendium

The template experiment is expected to have the same genes as the compendium experiment. Genes that are in the template experiment but not in the compendium are removed. Genes that are in the compendium but missing in the template experiment are added and the gene expression value is set to the median gene expression value of that gene across the samples in the compendium. Additionally, the template values are expected to come from the same distribution as the compendium dataset (i.e. both the template and compendium expression measurements are estimated counts). This is necessary since SOPHIE applies the same scale factor used to normalize the compendium to normalize the template experiment. If the template has a different range of expression values, then the scaling will result in outliers (i.e. values greater than 1) which the cross entropy loss in the VAE will not handle.

In [7]:
simulate_expression_data.process_template_experiment(
    raw_template_filename,
    raw_compendium_filename,
    scaler_transform_filename,
    mapped_template_filename,
    normalized_template_filename,
)

(72, 17755)
(49651, 17755)


## Simulate data

In [8]:
# Run simulation
simulate_expression_data.embed_shift_template_experiment(
    normalized_compendium_filename,
    normalized_template_filename,
    vae_model_dir,
    project_id,
    scaler_transform_filename,
    local_dir,
    latent_dim,
    num_simulated,
    simulated_data_dir,
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## Process template and simulated experiments

* Remove samples not required for comparison
* Make sure ordering of samples matches metadata for proper comparison
* Make sure values are cast as integers if using DESeq
* Filter lowly expressed genes if using DESeq

In [9]:
## Update simulated dir
if not os.path.exists(template_process_samples_filename):
    template_process_samples_filename = None

if de_method == "deseq":
    # Process template data
    stats.process_samples_for_DESeq(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        count_threshold,
        template_process_samples_filename,
    )

    # Process simulated data
    for i in range(num_simulated):
        simulated_filename = os.path.join(
            simulated_data_dir,
            f"selected_simulated_data_{project_id}_{i}.tsv",
        )
        out_simulated_filename = os.path.join(
            simulated_data_dir,
            f"selected_simulated_data_{project_id}_{i}_processed.tsv",
        )
        stats.process_samples_for_DESeq(
            simulated_filename,
            template_DE_grouping_filename,
            out_simulated_filename,
            count_threshold,
            template_process_samples_filename,
        )
else:
    stats.process_samples_for_limma(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        template_process_samples_filename,
    )

    for i in range(num_simulated):
        simulated_filename = os.path.join(
            simulated_data_dir,
            f"selected_simulated_data_{project_id}_{i}.tsv",
        )
        stats.process_samples_for_limma(
            simulated_filename,
            template_DE_grouping_filename,
            None,
            template_process_samples_filename,
        )

sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly


## Differential expression analysis

In [10]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [11]:
%%R -i template_DE_grouping_filename -i project_id -i processed_template_filename -i local_dir -i base_dir -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# File created: "<local_dir>/DE_stats/DE_stats_template_data_<project_id>_real.txt"
if (de_method == "deseq"){
    get_DE_stats_DESeq(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}
else{
    get_DE_stats_limma(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:limma’:

    plotMA


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply

[1] "Checking sample ordering..."
[1] TRUE


In [12]:
%%R -i template_DE_grouping_filename -i project_id -i base_dir -i simulated_data_dir -i num_simulated -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# Files created: "<local_dir>/DE_stats/DE_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_simulated-1)){
    simulated_data_filename <- paste(
        simulated_data_dir,
        "/selected_simulated_data_",
        project_id,
        "_",
        i,
        "_processed.tsv",
        sep = ""
    )
    if (de_method == "deseq"){
        get_DE_stats_DESeq(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
    }
    else {
        get_DE_stats_limma(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
        }
    }

[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE


## Rank genes

Genes are ranked by their "generic-ness" - how frequently these genes are changed across the simulated experiments using user-specific test statistic provided in the `col_to_rank_genes` params (i.e. log2 fold change).

In [13]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)

# Added
if de_method == "deseq":
    logFC_name = "log2FoldChange"
    pvalue_name = "padj"
else:
    logFC_name = "logFC"
    pvalue_name = "adj.P.Val"

template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_simulated,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


## Summary table

* Gene ID: Gene identifier (hgnc symbols for human data or PA number for *P. aeruginosa* data)
* (Real): Statistics for template experiment
* (Simulated): Statistics across simulated experiments
* Number of experiments: Number of simulated experiments
* Z-score: High z-score indicates that gene is more changed in template compared to the null set of simulated experiments (high z-score = highly specific to template experiment). These z-scores are true standard scores using mean and standard deviation. The calculation for the z-score for a given gene is
    
$$
\frac{\text{log}_2 \text{fold change of the gene in the template experiment} - mean(\text{log}_2 \text{fold change of the gene in simulated experiments)}}{variance(\text{log}_2 \text{fold change of the gene in simulated experiments)}}
$$

The range of this z-score will vary depending on the number of simulated experiments, so the number of simulated experiments should be held constant if the user is performing multiple SOPHIE runs or if they're comparing to previous SOPHIE runs performed by someone else.

* Percentile (simulated): percentile rank of the median(abs(log fold change)). So its the median absolute change for that gene across the 25 simulated experiments that is then converted to a percentile rank from 0 - 100. Where a higher percentile indicates that the gene was highly changed frequently and would suggest that the gene is more commonly DE.
* Percent DE (simulated): the fraction of the simulated experiments in which that gene was found to be DE using (log fold change > 1 and adjusted p-value < 0.05). _Note:_ you may find that many genes have a 0 fraction. This is because there is some compression that happens when pushing data through the VAE so the variance of the simulated experiments is lower compared to the real experiment. We are aware of this limitation in the VAE and are looking at how to improve the variance and biological signal captured by the VAE, however we were still able to demonstrate that for now the VAE is able to simulate realistic looking biological experiments in our previous [paper](https://academic.oup.com/gigascience/article/9/11/giaa117/5952607).


**Note:**
* If using DESeq, genes with NaN in only the `Adj P-value (Real)` column are those genes flagged because of the `cooksCutoff` parameter. The cook's distance as a diagnostic to tell if a single sample has a count which has a disproportionate impact on the log fold change and p-values. These genes are flagged with an NA in the pvalue and padj columns of the result table.

* If using DESeq with count threshold, some genes may not be present in all simulated experiments (i.e. the `Number of experiments (simulated)` will not equal the number of simulated experiments you specified in the beginning. This pre-filtering will lead to some genes found in few simulated experiments and so the background/null set for that gene is not robust. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

* If using DESeq without count threshold, some genes may still not be present in all simulated experiments (i.e. the `Number of experiments (simulated)`  will not equal the number of simulated experiments you specified in the beginning. If the gene is 0 expressed across all samples and thus automatically given an NA in `log fold change, adjusted p-value` columns. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

For more information you can read [DESeq FAQs](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA)

In [14]:
# Get summary table
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head(10)

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
POLR3G,POLR3G,0.350234,13020.0,0.524504,-0.524504,0.999029,5935.0,33.457375,0.0,0.045689,9.517811e-07,2,503073.088521
PIK3CD,PIK3CD,0.955033,11009.0,0.350935,-0.350935,0.999029,4614.0,26.009247,0.0,0.037304,2.226860e-06,2,140839.875928
PTPN9,PTPN9,0.223160,10164.0,0.307612,-0.307612,0.999029,54.0,0.298827,0.0,0.003154,2.402227e-06,2,126739.632698
HOXB13,HOXB13,0.997185,8847.0,0.243085,0.243085,0.999029,15174.0,85.549166,0.0,0.151866,1.495847e-06,2,60980.900725
ANXA2P3,ANXA2P3,0.497455,12342.0,0.457281,-0.457281,0.999029,2395.0,13.497970,0.0,0.023556,8.123901e-06,2,53388.838680
SLC22A2,SLC22A2,0.372011,16949.0,3.189778,-3.189778,0.999029,6865.0,38.700947,0.0,0.052124,5.929727e-05,2,52913.969052
ASIC3,ASIC3,0.707335,13360.0,0.576473,0.576473,0.999029,5732.0,32.312810,0.0,0.044370,2.337586e-05,2,22762.887990
CTNND2,CTNND2,0.077016,17069.0,3.958885,-3.958885,0.999029,16828.0,94.874831,0.0,0.225493,1.809854e-04,2,20628.145962
GRTP1.AS1,GRTP1.AS1,0.721660,16162.0,1.596353,1.596353,0.999029,12693.0,71.560668,0.0,0.104590,7.796695e-05,2,19133.269733
SSC4D,SSC4D,0.533783,12875.0,0.506314,-0.506314,0.999029,5464.0,30.801759,0.0,0.042628,2.940813e-05,2,15767.291034


In [15]:
summary_gene_ranks.isna().any()

Gene ID                                 False
Adj P-value (Real)                       True
Rank (Real)                              True
abs(log2FoldChange) (Real)               True
log2FoldChange (Real)                    True
Median adj p-value (simulated)           True
Rank (simulated)                         True
Percentile (simulated)                   True
Percent DE (simulated)                  False
Mean abs(log2FoldChange) (simulated)     True
Std deviation (simulated)                True
Number of experiments (simulated)       False
Z score                                  True
dtype: bool

In [16]:
summary_gene_ranks[summary_gene_ranks.isna().any(axis=1)]

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
LINC00317,LINC00317,0.717849,17046.0,3.732698,-3.732698,0.999164,7434.0,41.909111,0.0,0.056090,NaN,1,NaN
PRSS58,PRSS58,0.771559,16961.0,3.243114,3.243114,0.998942,1227.0,6.912494,0.0,0.015843,NaN,1,NaN
LINC01526,LINC01526,0.464434,16776.0,2.561850,2.561850,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN
LINC01049,LINC01049,0.804101,16775.0,2.557619,2.557619,0.998936,11109.0,62.629680,0.0,0.086090,NaN,1,NaN
CASC6,CASC6,0.997185,13982.0,0.677349,0.677349,0.998936,17234.5,97.166779,0.0,0.264498,NaN,1,NaN
CCT8L2,CCT8L2,0.997185,13860.0,0.654695,0.654695,0.998936,16181.0,91.226883,0.0,0.188830,NaN,1,NaN
CRYGA,CRYGA,0.997185,13784.0,0.643110,-0.643110,0.998936,16445.0,92.715381,0.0,0.201317,NaN,1,NaN
TSHB,TSHB,0.997185,13776.0,0.643109,-0.643109,0.998936,17594.0,99.193730,0.0,0.352999,NaN,1,NaN
LINC00911,LINC00911,0.979317,13079.0,0.530742,0.530742,0.998936,17153.0,96.707262,0.0,0.256651,NaN,1,NaN
LINC00845,LINC00845,0.997185,12890.0,0.507953,0.507953,0.999164,7434.0,41.909111,0.0,0.056090,NaN,1,NaN


In [17]:
# Save
summary_gene_ranks.to_csv(output_filename, sep="\t")